In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


* On a utilisé le site web **Imdb** pour collecter les données pour cela on a récupéré le contenu des tags HTML à partir du code source des pages qui nous intéressent c'est ce qu'a l'appele le **Scraping**

In [2]:
import bs4
import pandas as pd
import requests

#Collecter les informations des films

* On commence par collecter les informations sur les top 100 films sur le site 
( Title -  Genre	- IMDB Rating		- description)
* Aprés on va sauvgarder ce Dataset dans un fichier csv 

##Se connecter a la page 

In [18]:
url = 'https://www.imdb.com/search/title/?count=100&groups=top_1000&sort=user_rating'
def get_page_contents(url):
    page = requests.get(url, headers={"Accept-Language": "en-US"})
    return bs4.BeautifulSoup(page.text, "html.parser")

soup = get_page_contents(url)

In [20]:
d= soup.findAll('p', class_='text-muted')
description=[]
for i in range(len(d)):
  if i% 2 !=0 :
    description.append(soup.findAll('p', class_='text-muted')[i].text)


##Fonctions d'extraction

In [21]:
def numeric_value(movie, tag, class_=None, order=None):
    if order:
        if len(movie.findAll(tag, class_)) > 1:
            to_extract = movie.findAll(tag, class_)[order]['data-value']
        else:
            to_extract = None
    else:
        to_extract = movie.find(tag, class_)['data-value']

    return to_extract


def text_value(movie, tag, class_=None):
    if movie.find(tag, class_):
        return movie.find(tag, class_).text
    else:
        return


def nested_text_value(movie, tag_1, class_1, tag_2, class_2, order=None):
    if not order:
        return movie.find(tag_1, class_1).find(tag_2, class_2).text
    else:
        return [val.text for val in movie.find(tag_1, class_1).findAll(tag_2, class_2)[order]]


def extract_attribute(soup, tag_1, class_1='', tag_2='', class_2='',
                      text_attribute=True, order=None, nested=False):
    movies = soup.findAll('div', class_='lister-item-content')
    data_list = []
    for movie in movies:
        if text_attribute:
            if nested:
                data_list.append(nested_text_value(movie, tag_1, class_1, tag_2, class_2, order))
            else:
                data_list.append(text_value(movie, tag_1, class_1))
        else:
            data_list.append(numeric_value(movie, tag_1, class_1, order))

    return data_list

##application

In [22]:
titles = extract_attribute(soup, 'a')

In [23]:
genre = extract_attribute(soup, 'span', 'genre')

In [24]:
imdb_rating = extract_attribute(soup, 'div', 'inline-block ratings-imdb-rating', False)

In [25]:
df_dict = {'Title': titles,'Genre': genre, 'IMDB Rating': imdb_rating,'description':description}


###Resultat

In [26]:
df= pd.DataFrame(df_dict)
df

,Title,Genre,IMDB Rating,description
0,Dara of Jasenovac,"\nDrama, War",\n\n9.7\n,\n The film is set in the Nazi-occupied Cro...
1,The Shawshank Redemption,\nDrama,\n\n9.3\n,\n Two imprisoned men bond over a number of...
2,The Godfather,"\nCrime, Drama",\n\n9.2\n,\n An organized crime dynasty's aging patri...
3,The Dark Knight,"\nAction, Crime, Drama",\n\n9.0\n,\n When the menace known as the Joker wreak...
4,The Godfather: Part II,"\nCrime, Drama",\n\n9.0\n,\n The early life and career of Vito Corleo...
...,...,...,...,...
95,Eternal Sunshine of the Spotless Mind,"\nDrama, Romance, Sci-Fi",\n\n8.3\n,"\n When their relationship turns sour, a co..."
96,Amélie,"\nComedy, Romance",\n\n8.3\n,\n Amélie is an innocent and naive girl in ...
97,Snatch,"\nComedy, Crime",\n\n8.3\n,"\n Unscrupulous boxing promoters, violent b..."
98,Requiem for a Dream,\nDrama,\n\n8.3\n,\n The drug-induced utopias of four Coney I...


##Sauvgarder le Dataset

* Il faut tout d'abord creer un répértoire "projet_TM" sur votre Drive avant de creer le fichier CSV 

In [38]:
df.to_csv('/content/drive/My Drive/projet_TM/Data.csv' , encoding="UTF-8",sep =";" , index = False)

# Collecter les avis sur les films

* le 2éme Dataset contient les reviews sur chaque film dans le fichier précedent plus le titre et le score

##connection au pages des films

In [27]:
url="https://www.imdb.com/search/title/?count=100&groups=top_1000&sort=user_rating"

In [ ]:
# find all a-tags with class:None
movie_tags = soup.find_all('a', attrs={'class': None})

# filter the a-tags to get just the titles
movie_tags = [tag.attrs['href'] for tag in movie_tags 
              if tag.attrs['href'].startswith('/title') & tag.attrs['href'].endswith('/')]

# remove duplicate links
movie_tags = list(dict.fromkeys(movie_tags))

print("There are a total of " + str(len(movie_tags)) + " movie titles")
print("Displaying 10 titles")
movie_tags[:10]

In [29]:
# movie links
base_url = "https://www.imdb.com"
movie_links = [base_url + tag + 'reviews' for tag in movie_tags]
print("There are a total of " + str(len(movie_links)) + " movie user reviews")
print("Displaying 10 user reviews links")
movie_links[:10]

There are a total of 100 movie user reviews
Displaying 10 user reviews links


['https://www.imdb.com/title/tt10554232/reviews',
 'https://www.imdb.com/title/tt0111161/reviews',
 'https://www.imdb.com/title/tt0068646/reviews',
 'https://www.imdb.com/title/tt0468569/reviews',
 'https://www.imdb.com/title/tt0071562/reviews',
 'https://www.imdb.com/title/tt0050083/reviews',
 'https://www.imdb.com/title/tt0167260/reviews',
 'https://www.imdb.com/title/tt0110912/reviews',
 'https://www.imdb.com/title/tt0108052/reviews',
 'https://www.imdb.com/title/tt1375666/reviews']

##extraire les review et les scores 

In [30]:
# get a list of soup objects
movie_soups = [get_page_contents(link) for link in movie_links]



In [31]:
pages_reviews=[msoup.findAll('div', class_='lister-item-content') for msoup in movie_soups]

In [32]:
list_revews=[]
for page_reviews in pages_reviews:
  list_revews.append([rf.find('a','title').text for rf in page_reviews])


In [33]:
r_view=[]

for page_reviews in pages_reviews:
  r_view.append([rf.find('span').text for rf in page_reviews])

##transformer en dataframe

In [34]:
import numpy as np

l1=[]
l2=[]
for i in range(len(titles)):
   for j in range(len(r_view[i])):
    z=(titles[i],r_view[i][j])
    l1.append(z) 
    df2 = pd.DataFrame(np.array(l1),columns=['titres', 'rates'])
   for j in range(len(list_revews[i])):
     z=(list_revews[i][j])
     l2.append(z) 
     df3 = pd.DataFrame(np.array(l2),columns=['avis'])
    
 

  

In [35]:
data_reviews=pd.concat([df2, df3], axis=1, ignore_index=True)
data_reviews.columns=["title","rate","review"]


### Resultat

In [36]:
data_reviews

,title,rate,review
0,Dara of Jasenovac,\n\n\n\n\n\n10/10\n,Extremely hartbreaking movie about one of the...
1,Dara of Jasenovac,\n\n\n\n\n\n10/10\n,Incredible.\n
2,Dara of Jasenovac,\n\n\n\n\n\n10/10\n,Facts\n
3,Dara of Jasenovac,\n\n\n\n\n\n10/10\n,Amazing Truth\n
4,Dara of Jasenovac,\n\n\n\n\n\n10/10\n,Dara iz Jasenovca\n
...,...,...,...
2491,American Beauty,\n\n\n\n\n\n9/10\n,The best of 1999.\n
2492,American Beauty,\n\n\n\n\n\n10/10\n,Look Closer...\n
2493,American Beauty,Jason Morales,LOOK CLOSER at this BEAUTiful AMERICAN film.\n
2494,American Beauty,\n\n\n\n\n\n10/10\n,The Beautiful Truth\n


##Sauvgarder le resultat

In [37]:
data_reviews.to_csv('/content/drive/My Drive/projet_TM/Data_Reviews.csv' , encoding="UTF-8",sep =";" , index = False)

*  Voilà ! les données sont prêtes à manipuler